# STEP 0: Install dependencies

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain-google-genai openai




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 6.4 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass #to secure api key
from langgraph.graph import StateGraph ,END #save progress of the graph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage



# Secure Gemini API Key input

In [3]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API key: ")

Enter your Gemini API key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model ="models/gemini-1.5-flash-latest" , temperature =0.2) #temperature controls randomness of a model
# more temperatature = more randomness

# STEP 3: Node to ask user for symptom

In [5]:
def get_symptom(state: dict) ->dict:
  symptom = input("Welcome to the hospital, Please neter your symptom")
  state["symptom"] = symptom
  return state #state is the statement the user entered

# STEP 4: Node to classify the symptom

In [33]:
def classify_symptom(state: dict) ->dict:
  prompt = (
      "You are a helpful medical assistent, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency\n -Mental Health\n"
      f"Symptom: {state['symptom']}\n"
      "Respond only with one word: General , Emergency or Mental health"
      "#Example : input:I have fever , Output: General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as: (category)") #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [34]:
def symptom_router(state: dict) -> dict:
  cat = state["category"].lower()
  if cat == "general":
    return "general"
  elif cat == "emergency":
    return "emergency"
  elif cat == "mental health":
    return "mental health"
  else:
    return "general"

# STEP 6: Category-specific response nodes


In [35]:
def general_node( state: dict) -> dict:
  state["answer"] = f" ' {state['symptom']}' :seems general : directing to general ward for consulting a doctor "
  return state

def emergency_node(state: dict) -> dict:
  state["answer"] = f" ' {state['symptom']}' :seems emergency : seeking immeadiate help "
  return state

def mental_health_node(state: dict) -> dict:
  state["answer"] = f" ' {state['symptom']}' :seems like a mental health concern: directing to mental health ward for consulting a therapist "
  return state

# STEP 7: Build LangGraph


In [36]:
#to stitch all these nodes togeather
builder = StateGraph(dict)
builder.set_entry_point("get_symptom")  #entry point
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental health", mental_health_node)


#defining edges
builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router,{
      "general": "general",
      "emergency" : "emergency",
      "mental health": "mental health"
})
builder.add_edge( "general" ,END)
builder.add_edge( "emergency",END)
builder.add_edge( "mental health",END)




# STEP 8: Compile and invoke the graph


In [39]:
graph = builder.compile()
final_state = graph.invoke({})
print("Final Output \n")
print(final_state["answer"])

Welcome to the hospital, Please neter your symptom I an having sinus infection again
LLM classifies the symptom as: (category)
Final Output 

 '  I an having sinus infection again' :seems general : directing to general ward for consulting a doctor 
